In [29]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [30]:
pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cpu

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")



tensor([1.], device='mps:0')


In [32]:
# Define parameters
alpha = 1 / 3  # Thermal diffusivity constant
a, b = 2, 2  # Plate dimensions
n_epochs = 10000
n_points = 2000

In [33]:
def initial_temperature(x, y):
    # Normalized initial condition: 50 for y <= 1, else 0 
    return torch.where(y <= 1.0, torch.tensor(1.0), torch.tensor(0.0))

In [34]:
# Analytical solution for comparison
def analytical_solution(x, y, t, terms=50):
    u = 0
    for m in range(1, terms + 1):
        for n in range(1, terms + 1):
            coeff = 200 / (np.pi**2 * m * n) * (1 + (-1)**(m + 1)) * (1 - np.cos(n * np.pi / 2))
            lambda_mn = (np.pi / 6) * np.sqrt(m**2 + n**2)
            u += coeff * np.sin(m * np.pi * x / a) * np.sin(n * np.pi * y / b) * np.exp(-lambda_mn**2 * t)
    return 50 * u  # Scale back to the actual temperature


In [35]:
# Define PINN model
class PINN2DHeat(nn.Module):
    def __init__(self):
        super(PINN2DHeat, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 128),  # Input: x, y, t
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 1)  # Output: u(x, y, t)
        )

    def forward(self, x, y, t):
        inputs = torch.cat([x, y, t], dim=1)
        return self.net(inputs)

In [36]:
# Xavier Initialization
"""
def xavier_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)
        """


'\ndef xavier_init(m):\n    if isinstance(m, nn.Linear):\n        nn.init.xavier_normal_(m.weight)\n        nn.init.zeros_(m.bias)\n        '

In [37]:
# Define the PDE residual
def heat_equation_residual(model, x, y, t, alpha):
    x.requires_grad_(True)
    y.requires_grad_(True)
    t.requires_grad_(True)
    u = model(x, y, t)
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True)[0]
    return u_t - alpha * (u_xx + u_yy)

In [38]:
# Boundary and initial condition losses
def initial_condition_loss(model, x, y):
    t_zero = torch.zeros_like(x)
    u_pred = model(x, y, t_zero)
    u_true = initial_temperature(x, y)
    return torch.mean((u_pred - u_true)**2)

In [39]:
def boundary_condition_loss(model):
    """Loss enforcing boundary conditions u(x, y, t) = 0 on edges."""
    x_edge = torch.linspace(0, a, 100).reshape(-1, 1)
    y_edge = torch.linspace(0, b, 100).reshape(-1, 1)
    t_edge = torch.linspace(0, 5, 100).reshape(-1, 1)

    # Evaluate on all edges
    u_pred_top = model(x_edge,  b * torch.ones_like(y_edge), t_edge)
    u_pred_bottom = model(x_edge, 0 * torch.ones_like(y_edge), t_edge)
    u_pred_left = model(0 * torch.ones_like(x_edge), y_edge, t_edge)
    u_pred_right = model(a * torch.ones_like(x_edge), y_edge, t_edge)

    return torch.mean(u_pred_top**2 + u_pred_bottom**2 + u_pred_left**2 + u_pred_right**2)

In [40]:
# Training function
def train_pinn(model, optimizer, n_epochs, n_points):
    loss_history = []
    for epoch in range(n_epochs):

        x = torch.rand((n_points, 1)) * a
        y = torch.rand((n_points, 1)) * b
        t = torch.rand((n_points, 1)) * 5
        
        loss_pde = torch.mean(heat_equation_residual(model, x, y, t, alpha)**2)
     
        x_init = torch.rand((n_points // 2, 1)) * a
        y_init = torch.rand((n_points // 2, 1)) * b
        loss_ic = initial_condition_loss(model, x_init, y_init)

        loss_bc = boundary_condition_loss(model)

        loss = 1.0 * loss_pde + 1.0 * loss_ic + 1.0 * loss_bc

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()

        loss_history.append(loss.item())
        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}")
    return loss_history

In [ ]:
model = PINN2DHeat()
# model.apply(xavier_init)
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2000, gamma=0.5)
# loss_history = train_pinn(model, optimizer, scheduler, n_epochs, n_points)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
loss_history = train_pinn(model, optimizer, n_epochs, n_points)

Epoch 0, Loss: 0.673442
Epoch 500, Loss: 0.036626
Epoch 1000, Loss: 0.024876
Epoch 1500, Loss: 0.030181
Epoch 2000, Loss: 0.021080
Epoch 2500, Loss: 0.025748
Epoch 3000, Loss: 0.016890
Epoch 3500, Loss: 0.020463
Epoch 4000, Loss: 0.055620
Epoch 4500, Loss: 0.016328
Epoch 5000, Loss: 0.019794
Epoch 5500, Loss: 0.016113
Epoch 6000, Loss: 0.017813
Epoch 6500, Loss: 0.033589
Epoch 7000, Loss: 0.022713
Epoch 7500, Loss: 0.021294


In [ ]:

plt.plot(loss_history)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()


In [ ]:
time_steps = [1, 2, 3, 4, 5]
x_vals = torch.linspace(0, a, 100).reshape(-1, 1)
y_vals = torch.linspace(0, b, 100).reshape(-1, 1)
X, Y = torch.meshgrid(x_vals.squeeze(), y_vals.squeeze())

fig, axes = plt.subplots(len(time_steps), 3, figsize=(15, 20))

for i, t_fixed in enumerate(time_steps):
    t_tensor = torch.full_like(X.reshape(-1, 1), t_fixed)
    u_pred = model(X.reshape(-1, 1), Y.reshape(-1, 1), t_tensor).detach().numpy().reshape(-1, 1) * 50
    u_exact = analytical_solution(X.numpy(), Y.numpy(), t_fixed, terms=50).reshape(-1, 1)
    error = np.abs(u_pred - u_exact)

    axes[i, 0].imshow(u_pred.reshape(100, 100), extent=[0, a, 0, b], origin='lower', cmap='hot')
    axes[i, 0].set_title(f"PINN Solution at t = {t_fixed}")

    axes[i, 1].imshow(u_exact.reshape(100, 100), extent=[0, a, 0, b], origin='lower', cmap='hot')
    axes[i, 1].set_title(f"Analytical Solution at t = {t_fixed}")

    axes[i, 2].imshow(error.reshape(100, 100), extent=[0, a, 0, b], origin='lower', cmap='viridis')
    axes[i, 2].set_title(f"Error at t = {t_fixed}")

plt.tight_layout()
plt.show()